In [ ]:
import os
import re
import glob
import shutil
import urllib
import requests

def get_files(root_dir, re_expr='.*'):
    def _chk_re_expr(string):
        return 

    file_list = []
    for dir_n, _, fn_list in os.walk(root_dir):
        if fn_list:
            cur_fs = [os.path.join(dir_n, f) for f in fn_list]
            cur_fs = [f for f in cur_fs if bool(re.fullmatch(re_expr, f))]
            file_list += cur_fs
    file_list = sorted(file_list)
    return file_list

In [ ]:
# OTB-100
from lxml import html, etree
html = etree.parse(web, etree.HTMLParser())
result = etree.tostring(html)
result = html.xpath('//a[contains(@href, "seq/")]')

base_url = 'http://cvlab.hanyang.ac.kr/tracker_benchmark/'
folder = os.path.expanduser('~/Downloads/OTB100/')
if os.path.isdir(folder):
    shutil.rmtree(folder)
os.makedirs(folder)
for r in result:
    url = base_url + r.attrib['href']
    dst = os.path.join(folder, url.split('/')[-1])
    
    urllib.request.urlretrieve(url, dst)

In [ ]:
# VOT
folder = os.path.expanduser('/Data/VOT/2018lt_zip/')
os.makedirs(folder, exist_ok=True)

log = '/home/tangliyao/Research/vot-toolkit/vot-2018/log_2018lt'
with open(log, 'r') as f:
    log = f.read().split('\n\n')
log = [l.strip('\n ') for l in log if l.strip('\n ')]

skip_seq = set([])
for seq in log:
    seq = [i.strip('\n ') for i in seq.split('\n') if i.strip('\n ')]
    seq = [i for i in seq if '...' in i or 'url' in i]
    
    seq_n = seq[0].split('"')[1]
    ann_url = seq[1].split('=')[-1].strip()
    img_url = seq[2].split('=')[-1].strip()
    
    if seq_n in skip_seq:
        continue

    print('downloading', seq_n)
    dst_folder = os.path.join(folder, seq_n)
    if os.path.isdir(dst_folder):
        shutil.rmtree(dst_folder)
    os.makedirs(dst_folder)
    
    for url in [ann_url, img_url]:
        cnt = 0
        dst = os.path.join(dst_folder, url.split('/')[-1])
        print('  =>', dst)

        while cnt < 100: # keep trying
            try:
                # urllib.request.urlretrieve(url, dst)
                os.system('wget %s -O %s' % (url, dst))  # using 'wget'
            except (urllib.error.URLError, urllib.error.HTTPError) as e:
                err_msg = e.reason
            except:
                err_msg = sys.exc_info()[0]
            else:  # success execution
                time.sleep(1)
                break
            finally:  # clean-up actions
                pass
            # some error occured
            print('\t%s - retrying ...' % err_msg, cnt)
            if os.path.isfile(dst):
                os.remove(dst)
            cnt += 1

In [ ]:
# check data
import hashlib
def get_md5(fname, chunk_size=1024*10):
    chunk_size = int(chunk_size)
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

data_dir = '/Data/VOT/2018lt_win'
file_list = get_files(data_dir, '.*\.zip')

output_file = 'md5sum'
with open(os.path.join(data_dir, output_file), 'w') as output_file:
    for f in file_list:
        md5 = get_md5(f)
        output_file.write('%s: %s\n' % (f, md5))

In [ ]:
# uncompress
import zipfile
data_dir = '/Data/VOT/2018lt_zip'
file_list = get_files(data_dir, '.*\.zip')

for zip_f in file_list:
    with zipfile.ZipFile(zip_f, 'r') as zip_ref:
        target_dir = os.path.dirname(zip_f) # parent dir to uncrompress
        target_dir = target_dir.replace('2018lt_zip', '2018lt')
        
        print(target_dir)
        zip_ref.extractall(target_dir)